In [ ]:
import pandas as pd
import numpy as np
import boto3
import configparser
import psycopg2

# Leemos archivo de configuración y credenciales

In [ ]:
config = configparser.ConfigParser()
config.read('config_P.cfg')

In [ ]:
aws_rds_conn = boto3.client('rds', aws_access_key_id=config.get('IAM', 'ACCESS_KEY'),
                            aws_secret_access_key=config.get('IAM', 'SECRET_KEY'),
                            region_name='us-east-2')

### Verificamo instancias disponibles para el usuario

In [ ]:
rds_instances_ids = []

aws_response = aws_rds_conn.describe_db_instances()

for item in aws_response['DBInstances']:
    rds_instances_ids.append(item['DBInstanceIdentifier'])
    
print(f"Instancias disponibles: {rds_instances_ids}")

### Creamos instancia de base de datos en AWS - Postgres

In [ ]:
try:
    response = aws_rds_conn.create_db_instance(
                            DBInstanceIdentifier=config.get('ECOMERCE', 'DB_INSTANCE_ID'),
                            DBName=config.get('ECOMERCE', 'DB_NAME'),
                            MasterUsername=config.get('ECOMERCE', 'DB_USERNAME'),
                            MasterUserPassword=config.get('ECOMERCE', 'DB_PASSWORD'),
                            Port=int(config.get('ECOMERCE', 'DB_PORT')),
                            DBInstanceClass='db.t3.micro',
                            Engine='postgres',
                            PubliclyAccessible=True,
                            AllocatedStorage=20,
                            VpcSecurityGroupIds=[config.get('VPC', 'SECURITY_GROUP')]
                        )
    print(response)
except aws_rds_conn.exceptions.DBInstanceAlreadyExistsFault:
    print('La instancia ya existe')
except Exception as ex:
    print('Error!!!', ex)

### Obtenemos el hostname de la instancia

In [ ]:
try: 
    instance = aws_rds_conn.describe_db_instances(DBInstanceIdentifier=config.get('ECOMERCE','DB_INSTANCE_ID'))
    RDS_HOSTNAME = instance.get('DBInstances')[0].get('Endpoint').get('Address')
    print(RDS_HOSTNAME)
except Exception as ex:
    print('Error!!!',ex)

### Nos conectamos a la base de datos y creamos las tablas

In [ ]:
import sql_queries_P

In [ ]:
try:
    db_pg_conn = psycopg2.connect(
                        database=config.get('ECOMERCE', 'DB_NAME'),
                        user=config.get('ECOMERCE', 'DB_USERNAME'),
                        password=config.get('ECOMERCE', 'DB_PASSWORD'),
                        port=config.get('ECOMERCE', 'DB_PORT'),
                        host=RDS_HOSTNAME)
    cursor = db_pg_conn.cursor()
    cursor.execute(sql_queries_P.DDL_QUERY)
    db_pg_conn.commit()
except Exception as ex:
    print('Error!!!!', ex)